In [14]:
import netCDF4
from netCDF4 import Dataset
import os
from os.path import dirname
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib 
import matplotlib.pyplot as plt
from math import pi
from numpy import cos,sin
from scipy.spatial import cKDTree
from numpy import absolute as abs
import numpy.ma as ma
from ipywidgets import interact, interactive, fixed, interact_manual

In [2]:
## Chose folder, resolution and dataset

resolution = '10m' # Change border resolution for the maps. Insert value in meters (e.g. '110m')
folder = dirname(dirname(os.getcwd())) + '\\Data_WQ\\' #The folder where the datasets are

'''
Choose the dataset:
    1: CHL-model
    2: CHL-satellite
    3: DOXYL-model
    4: NITR-model
    5: PHOS-model
    6: SPM-satellite
'''
num_dataset = 1 #Change the number here

In [3]:
## Open dataset and store variables in var_values dictionary. Units are in var_units dictionary. Names in var_names

# Open dataset
if num_dataset == 2:
    file = folder + "dataset-CHL-satellite-daily.nc"
    selected_variable = 'CHL'
    text = 'Mass concentration of chlorophyll-a in sea water - satellite'
elif num_dataset == 3:
    file = folder + "dataset-DOXYL-model-daily.nc"
    selected_variable = 'o2'
    text = 'Mole concentration of dissolved molecular oxygen in sea water - model'
elif num_dataset == 4:
    file = folder + "dataset-NITR-model-daily.nc"
    selected_variable = 'no3'
    text = 'Mole concentration of nitrate in sea water - model'
elif num_dataset == 5:
    file = folder + "dataset-PHOS-model-daily.nc"
    selected_variable = 'po4'
    text = 'Mole concentration of phosphate in sea water - model'
elif num_dataset == 6:
    file = folder + "dataset-SPM-satellite-monthly.nc"
    selected_variable = 'SPM'
    text = 'Mass concentration of inorganic suspended matter in sea water - satellite'
else:
    file = folder + "dataset-CHL-model-daily.nc"
    selected_variable = 'chl'
    text = 'Mass concentration of chlorophyll-a in sea water - model'

dataset = Dataset(file, "r")
print('Using dataset {}'.format(file))

# Save variables, units and names
var_values = {}
var_units = {}
var_names = {}
for k in dataset.variables.keys():
    # Standardize keys 'latitude' and 'longitude'
    if k == 'lat':
        k_new = 'latitude'
    elif k == 'lon':
        k_new = 'longitude'
    else:
        k_new = k
  
    var_units[k_new] = dataset.variables[k].units # Save units
    var_names[k_new] = dataset.variables[k].long_name # Save name
    if 'time' in var_names[k_new]: #Save time
        var_values[k_new] = netCDF4.num2date(dataset.variables[k][:],var_units[k_new], only_use_cftime_datetimes=False, only_use_python_datetimes = True)
    else:
        var_values[k_new] = dataset.variables[k][:]
    
    # Remove single-dimensional entries
    var_values[k_new] = np.squeeze(var_values[k_new])
        
# Close dataset
dataset.close()

Using dataset C:\Users\Rafael\Google Drive\Rafi\Delft\Mathematical Data Science\Project\Data_WQ\dataset-CHL-model-daily.nc


In [20]:
var_values[selected_variable].std(axis=0)

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [0.6748787021128767, 0.7045210888243567, 0.6859229155353349, ...,
         --, --, --],
        [0.6705110871385946, 0.6831806251973709, 0.6461629688869077, ...,
         --, --, --],
        [0.7669922327903136, 0.7352047260727398, 0.6719659952994583, ...,
         --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=-32768)

In [21]:
## Plot

def plot_map_summary(sel):
    if sel == 'Max':
        var_plot = var_values[selected_variable].max(axis=0)
    elif sel == 'Min':
        var_plot = var_values[selected_variable].min(axis=0)
    elif sel == 'Mean':
        var_plot = var_values[selected_variable].mean(axis=0)
    elif sel == 'Deviation':
        var_plot = var_values[selected_variable].std(axis=0)
    else:
        var_plot = var_values[selected_variable].max(axis=0)
    text_plot = sel + " of " + text 

    # Initialize plot
    matplotlib.rcParams['figure.figsize'] = (10,10) 

    # Initialize map
    proj=ccrs.Mercator()
    m = plt.axes(projection=proj)

    # Format map
    m.stock_img()
    #m.coastlines(resolution='110m')
    m.coastlines(resolution=resolution)
    m.add_feature(cfeature.BORDERS)
    gl=m.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabels_top = False
    gl.ylabels_right = False

    # Plot data
    plt.contourf(var_values['longitude'], var_values['latitude'], var_plot, 60,
                 transform=ccrs.PlateCarree())

    # Add Colorbar
    cbar = plt.colorbar()
    cbar.set_label(var_units[selected_variable])

    # Add Title
    plt.title(text_plot)
    plt.show()

In [22]:
interact_manual(plot_map_summary, sel=['Max', 'Min', 'Mean', 'Deviation'])

interactive(children=(Dropdown(description='sel', options=('Max', 'Min', 'Mean', 'Deviation'), value='Max'), B…

<function __main__.plot_map_summary(sel)>